# iCompare Demo on select asteroids

This notebook has been developed as a demonstration of the iCompare package and its functionality. The full aims are listed in the README of the GitHub repository, found [here](https://github.com/maria8ch/iCompare), and summarized below.

There are roughly 5 integrators that are most commonly in use in Solar System dynamics community: JPL Horizons, OpenOrb, FindOrb, OrbFit, and ADAM. They have never been fully compared and tested, which may prove problematic for certain use cases. We are developing automated code that propagates asteroids using all of these integrators, compares their outputs, and presents it in some visually understandable way (e.g., a colourcoded matrix w. the asteroid on the Y axis and the integrator on the X axis).

## Procedural information
Our procedures are as follows: to initiate the integrators, we are using orbital elements provided by JPL Horizons in cometary (COM) format. We are using an integration epoch from 2010/01/01 to 2020/01/01, midnights; observing location is defined to be Ill, the Gemini South telescope. 

## Current issues:
Some of the integrators contain certain features that affect the capabilities of iCompare. For example, FindOrb currently does not support orbital elements in cometary format as an intializer (it uses state vectors instead). As such, it is not yet part of the iCompare suite. Please contact the respective teams via their communication channels for resolution of those types of issues:

* JPL Horizons: https://ssd.jpl.nasa.gov/horizons.cgi
* OpenOrb: https://github.com/oorb/oorb
* FindOrb: https://www.projectpluto.com/find_orb.htm
* OrbFit: http://adams.dm.unipi.it/orbfit/
* ADAM: https://github.com/B612-Asteroid-Institute/adam_home


## Imports and Initialization of package
The next portion of this notebook shows two examples of iCompare functionality: first on a typical MBA: 202930, and second on a larger subset of asteroids.

In [1]:
# necessary to ensure that updates to icompare are implemented with every re-run
%load_ext autoreload
%autoreload 2
import icompare as ic

## Example 1:  Asteroid 202930
This test shows the initialization of iCompare's functions for a single asteroid.
### Ephemeris extraction
To compare all available integrator ephemerides, the following variables must be specified, in string format, in the following order: object ID; the beginning and end of the integration arc epoch; and the observatory code. iCompare takes these inputs and provides two outputs, the JPL Elements ('el_jpl'), and the ephemerides results from each integrator ('results').

The cells below show proper initialization via the get_ephems( ) function and its outputs:

In [2]:
# initialization for example 1
obj_id = '202930'
start = '2010-01-01T00:00:00'
stop = '2020-01-01T00:00:01'
obs = 'I11'

In [3]:
el_jpl, results = ic.get_ephems(obj_id, start, stop, obs)

Elements done
JPL Horizons done
OpenOrb done
OrbFit done


1       53.91
2       32.62
3        6.69
4       36.05
        ...  
3648    56.16
3649     8.19
3650    15.92
3651    19.34
3652    18.44
Name: 7, Length: 3653, dtype: float64', which is not in range [0,60). Treating as 0 sec, +1 min [astropy.coordinates.angle_utilities]


In [4]:
el_jpl

targetname,datetime_jd,datetime_str,H,G,e,q,incl,Omega,w,Tp_jd,n,M,nu,a,Q,P
---,d,---,mag,---,---,AU,deg,deg,deg,d,deg / d,deg,deg,AU,AU,d
str26,float64,str30,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
202930,2455197.5,A.D. 2010-Jan-01 00:00:00.0000,16.7,0.15,0.128838465302325,2.370222773858959,4.55987116003829,346.4356563687987,8.2675176791458,2455990.023345738,0.2196182167740337,185.9474360573336,184.6298950202226,2.720761511447487,3.071300249036016,1639.208282846617


In [5]:
results

{'JPL Horizons': (<Quantity [[196.02761, 196.21191, 196.39288, ..., 254.07225, 254.53002,
              254.98785],
             [ -7.93628,  -8.03164,  -8.12573, ..., -26.07109, -26.12204,
              -26.17179]] deg>,
  <Quantity [[22.501, 22.492, 22.482, ..., 22.28 , 22.284, 22.288]] mag>),
 'OpenOrb': (<Quantity [[196.02760876, 196.21190699, 196.3928799 , ..., 254.07229114,
              254.53005843, 254.98788509],
             [ -7.9362818 ,  -8.03164084,  -8.12572828, ..., -26.07109631,
              -26.12204612, -26.17179442]] deg>,
  <Quantity [[22.50220877, 22.49281867, 22.48323986, ..., 22.29579726,
              22.29946906, 22.30292282]] mag>),
 'OrbFit': (<Quantity [[196.02760833, 196.21190833, 196.39288333, ..., 254.07223333,
              254.53      , 254.987825  ],
             [ -7.93628333,  -8.03164167,  -8.12572778, ..., -26.07108889,
              -26.12203889, -26.17178889]] deg>,
  <Quantity [[22.5, 22.5, 22.5, ..., 22.3, 22.3, 22.3]] mag>)}

### Great circle distance and other comparisons
iCompare also contains the comparison function gc_dist( ) that returns great circle distance ('rsep') between integrator outputs with respect to a given reference. Currently, this reference is the JPL Horizons outputs. It also provides the difference in computed magnitudes ('rmag'). In the future, we intend to expand this suite to other comparative features. Below is the appropriate use of the function and its outputs:

In [6]:
rsep, rmag = ic.gc_dist(results, reference="JPL Horizons")

In [7]:
rsep

{'OpenOrb': array([0.00785716, 0.01113804, 0.00619812, ..., 0.13494724, 0.12616712,
        0.11447877]),
 'OrbFit': array([0.01339081, 0.00844377, 0.01432212, ..., 0.0540432 , 0.06476943,
        0.08087179])}

In [8]:
rmag

{'OpenOrb': <Quantity [[0.00120877, 0.00081867, 0.00123986, ..., 0.01579726,
             0.01546906, 0.01492282]] mag>,
 'OrbFit': <Quantity [[-0.001,  0.008,  0.018, ...,  0.02 ,  0.016,  0.012]] mag>}

### Statistical summary and tables
Finally, iCompare can summarize the major statistical points in a simple table throught the table_stats( ) function. The cell below demonstrates this function as used on the great circle distance separation. The output is a dataframe table with the mean, median and maximum separations:

In [9]:
stats = ic.table_stats(rsep)
stats

integrator        
OpenOrb     mean      0.041671
            median    0.025782
            max       0.154741
OrbFit      mean      0.025471
            median    0.021910
            max       0.082840
dtype: float64

### Colour Map options
Additionally, this table can be colour-coded. There are two colour-coding options: the default map, and the red-green colourblind map. The following colour/value matches have been adopted for colour-coding:

#### Default colour map
value <= 0.05" -> green (good)

value <= 0.2" -> yellow (ok)

value <= 0.6" -> orange (not good but may work)

otherwise -> red (bad)

#### Green/red colourblind-friendly colour map
value <= 0.05" -> no colour (good)

value <= 0.2" -> blue (ok)

value <= 0.6" -> orange (not good but may work)

otherwise -> red (bad)

In [10]:
ic.plot(data=stats, cmap='default')

In [11]:
ic.plot(data=stats, cmap='rg_friendly')

## Example 2: Default objects
iCompare has a built-in set of 22 default objects for testing purposes. These objects were chosen to showcase the application range of each integrator and the limitations they may have. Some of these objects are MBA's, and some are part of stranger classes, such as impactors and TNO's. 

Here is a list of all default objects: 202930, 110, 887, 6489, 176014, 136199, 15760, 2018VP1, 101955, 99942, 5261, 624, 588, 10199, 136472, 136199, 2008TC3, 2020VT4, 433, 367943, 2020AV2, A/2017 U1

For convenience, iCompare has a calc_all( ) function that allows the user to run all previously showcased functions on all of the objects at once:

In [ ]:
result = ic.calc_all()

Elements done
JPL Horizons done
OpenOrb done


1       53.91
2       32.62
3        6.69
4       36.05
        ...  
3648    56.16
3649     8.19
3650    15.92
3651    19.34
3652    18.44
Name: 7, Length: 3653, dtype: float64', which is not in range [0,60). Treating as 0 sec, +1 min [astropy.coordinates.angle_utilities]


OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done


1       26.21
2       21.23
3       10.25
4       53.30
        ...  
3648    24.12
3649     1.42
3650    36.01
3651     7.90
3652    37.10
Name: 7, Length: 3653, dtype: float64', which is not in range [0,60). Treating as 0 sec, +1 min [astropy.coordinates.angle_utilities]


OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done


...

OrbFit done
Elements done
JPL Horizons done
OpenOrb done


1       12.30
2       52.51
3       55.72
4       22.16
        ...  
3648    35.74
3649     6.63
3650    17.99
3651    11.48
3652    48.79
Name: 7, Length: 3653, dtype: float64', which is not in range [0,60). Treating as 0 sec, +1 min [astropy.coordinates.angle_utilities]


OrbFit done
Elements done
JPL Horizons done
OpenOrb done


1       16.13
2       44.50
3       31.57
4       36.92
        ...  
3648    47.92
3649    17.47
3650    41.88
3651     1.01
3652    14.74
Name: 7, Length: 3653, dtype: float64', which is not in range [0,60). Treating as 0 sec, +1 min [astropy.coordinates.angle_utilities]


OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done


In [ ]:
result

As you can see the results are not easily parsible due to hightened volume. For this reason, the results of multi-object integration are best viewed in colour-coded tables via ic.plot( ):

In [ ]:
ic.plot(data=result, cmap='default')

In [ ]:
ic.plot(data=result, cmap='rg_friendly')

<font size="4"> If you have any questions, please open a pull request on [Github](https://github.com/maria8ch/iCompare). </font>